<a href="https://colab.research.google.com/github/s2akbarp/NWT-land-Cover-Classification/blob/main/Classification_2017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Shae Akbarpour https://github.com/s2akbarp

Using Neural network and mechine learning algorithm for classification of land covers in discontinous permafrost regions of NWT.
=============================================================
Here We used:
The World View(WV) 2 (2010) Image will be used to classify the primary land covers in NWT 
       1: Fen
       2:Permafrost plateaus
       3: Bogs
The train data 

The classification is done by three different methodologies:
       1: Random Forest 
       2: Support vector machine
       3: Logistic regression
The model wil be trained on 80% of data and tested on 20%. All the fitted models  will be validated by using a 10-fold cross validation on  traning set.
After fitting the model and selecting best model with higher accuracy, the model will be used to predict land covers on a WV3 (2017) imagery.


## Procedures
*   import data and training data
*   preprocess the data
*   build the models and functions
*   validate the trained model by 10-fold cross validation
*   evaluate the model on test set
*   classify 2017 imagery by using model with highest accuracy






In [ ]:
# Just used for google colab, if you are not using google colab comment this code
! pip install geopandas rasterio matplotlib descartes scikit-learn

Dataset
===================================
import:
1= shapefile of the training dataset which contain classification of land covers
2=  World veiew 2 imagery of the arctic region with 8 bands.

In [ ]:
from google.colab import files
uploaded = files.upload()

Import al the libraries used for classification

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gdal
import random
import math
import itertools
import rasterio
from rasterio.plot import adjust_band
import matplotlib.pyplot as plt
from rasterio.plot import reshape_as_raster, reshape_as_image
from rasterio.plot import show
from rasterio.windows import Window
import rasterio.features
import rasterio.warp
import rasterio.mask
from pyproj import Proj, transform
from tqdm import tqdm
from shapely.geometry import Polygon
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import numpy as np
from shapely.geometry import mapping
import os

Analyze the main world view 2 imagery.

In [ ]:
import os 
image_dir = '/content/drive/My Drive/classification/'

# find every file in the  directory
image_paths = [os.path.join(image_dir, '2010_WV2_WGS84.tiff') ]

image_wv2 = image_dir + '2010_WV2_WGS84.tiff'
dataset = rasterio.open(image_wv2, dtype='float32')
img_rows, img_cols = dataset.shape
img_bands =dataset.count
print(dataset.shape) # dimensions
print(dataset.count)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Read in the training labels image which represents 4 classes in the case study:


In [ ]:
train2010=gpd.read_file('/content/drive/My Drive/classification/train2010_scrs_wv2.shp')

geoms = train2010.geometry.values 

geometry = geoms[0] 
print(type(geometry))
print(geometry)

#  to GeoJSON format
from shapely.geometry import mapping
alltrains = [mapping(geometry)]
print(alltrains)


In this section:

1:Calculate NDVI and NDWI.
We will add these to variables to the model for classification as a feature.

2: stack all the latyes to array which contains all bands.

In [ ]:
# ignore  "divide by zero" or "divide by NaN"
np.seterr(divide='ignore', invalid='ignore')
wv2_2010=rasterio.open('/content/drive/My Drive/classification/2010_WV2_WGS84.tiff')
wv2_2010_img=wv2_2010.read()


#1 claculate ndvi and ndwi and save them in a directory: first we define the function and then calculate ndvi and ndwi
def calculate_ndvi(imagery):
    NIR_Band=imagery[7,:,:]
    Red_Band=imagery[5,:,:]
    ndvi = np.clip((NIR_Band.astype(np.float16) - Red_Band.astype(np.float16)) / (NIR_Band.astype(np.float16) + Red_Band.astype(np.float16)), -1,1)
    ndviDire=rasterio.open('../content/drive/My Drive/classification/newdata/newbands/ndvi.tiff', 'w', driver='Gtiff',width=wv2_2010.width,height=wv2_2010.height,count=1,crs=wv2_2010.crs,dtype='float16')
    ndviDire.write(ndvi,1)
    return ndvi

def calculate_ndwi(imagery):
    NIR_Band=imagery[7,:,:]
    Green_Band=imagery[3,:,:]
    ndwi = np.clip((Green_Band.astype(np.float16) - NIR_Band.astype(np.float16))/ (Green_Band.astype(np.float16)+ NIR_Band.astype(np.float16)), -1,1)
    ndwiDire=rasterio.open('../content/drive/My Drive/classification/newdata/newbands/ndwi.tif', 'w', driver='Gtiff',width=wv2_2010.width,height=wv2_2010.height,count=1,crs=wv2_2010.crs,dtype='float16')
    ndwiDire.write(ndwi,1)
    return ndwi




#  3: stack all the bands in the directory

    #call all the features in the directory
all_features = '/content/drive/My Drive/classification/newdata/newbands'
band_paths = [os.path.join(all_features, f) for f in os.listdir(all_features) if os.path.isfile(os.path.join(all_features, f))]
band_paths.sort()
band_paths
    
    
    #create a layer for writing all the raster bands in the directory
All_features = all_features + 'All_bands.tif'

with rasterio.open(band_paths[0]) as src0:
    meta = src0.meta
# Update  the number of layers

meta.update(count = len(band_paths))

# Read each layer and write it to stack
with rasterio.open(All_features, 'w', **meta) as dst:
    for id, layer in enumerate(band_paths, start=1):
        with rasterio.open(layer) as src1:
            dst.write_band(id, src1.read(1))


full_dataset_new = rasterio.open(All_features)
img_rows, img_cols = full_dataset_new.shape
img_bands = full_dataset_new.count
print(full_dataset_new.shape) # dimensions
print(full_dataset_new.count) # bands





In [ ]:
import rasterio as rio
train2010=rio.open("train2010.tif")
coundband_2010=train2010.count
train2010_img=train2010.read()
unique, counts = np.unique(train2010_img, return_counts=True)
#list(zip(unique, counts))
#fig, ax = plt.subplots(figsize=(10, 10))
# we then use these objects to draw-on and manipulate our plot
#ax.imshow(train2010_img[0])
train2010.crs



















ndvi=rasterio.open('/content/drive/My Drive/classification/newdata/ndvi.tiff')
ndwi=rasterio.open('/content/drive/My Drive/classification/newdata/ndwi.tif')
allbands=rasterio.open('/content/drive/My Drive/classification/newdata/allbands.tiff')

import rasterio

file_list = [ndvi, ndwi, allbands]

out_img = "stack.tif"
stack=rasterio.open('/content/drive/My Drive/classification/newdata/stack.tif')
print(stack.count)


out_meta = ndvi.meta.copy()
out_meta.update({"count": 10,"nodata": -10000})
with rasterio.open(out_img, 'w', **out_meta) as dest:
    for band_nr, src in enumerate(file_list, start=1):
        dest.write(src, band_nr, indexes=10)

stack=rasterio.open('/content/drive/My Drive/classification/newdata/stack.tif')




Visulize the imagery and new features

In [ ]:
from rasterio.plot import show
bands = np.array([5, 3, 2])
colors_image = wv2_2010_img[bands, :, :].astype(np.float64)

# rasters are in the format [bands, rows, cols] whereas images are typically [rows, cols, bands]
rasterio.plot.show(colors_image)
rasterio.plot.show(ndwi,cmap='RdYlGn')



#mask data

out_image, out_transform = mask(dataset, alltrains, crop=True)
out_image.shape






Now  get the pixels from the World view2 imagery as outlined in each traininig shapefile.
Our training data contains one main field:

a  field which contains class names in String datatype.


However, in order to pair up our vector data with our raster pixels, we will need a way of co-aligning the datasets in space which can be done by a function called mask in rasterio.

Let's Start the journey 
==========================
